In [3]:
!cd .. & ./setup.sh

/bin/sh: ./setup.sh: No such file or directory


# Finding the best set of parameters

In [4]:
from covidsimulation.regions.br_saopaulo import params as br_saopaulo_params, score_fn as br_saopaulo_score_fn, \
                                                sp_official_deaths
from covidsimulation.callibrate import callibrate_parameters
from covidsimulation import Parameters
from tqdm.notebook import tqdm


max_simulation_day = max(d[0] for d in sp_official_deaths) + 1


def set_d0_infections(p: Parameters, v):
    p.d0_infections = v


def set_population_1_seed_infections(p: Parameters, v):
    p.population_segments[1].seed_infections = v


callibrate_parameters(
    [
        (set_d0_infections, list(range(8000, 13000, 1000))),
        (set_population_1_seed_infections, (4, 50, 12)),
    ],
    br_saopaulo_score_fn,
    br_saopaulo_params,
    duration=max_simulation_day,
    simulation_size=20000,  # Recommended: simulation_size >= 200000
    n=4,  # Recommended: n > 8
    # tqdm=tqdm,  # uncomment to display progress
    )

array([[ 8000,    50],
       [ 9000,    50],
       [10000,    50],
       [11000,    50],
       [12000,    50],
       [ 8000,    12],
       [ 9000,    12],
       [10000,    12]])

# Showing the simulations that best fit history

In [8]:
from covidsimulation import run_simulations, plot

stats = run_simulations(
    sim_params=br_saopaulo_params, 
    simulate_capacity=True, 
    duration=60,
    number_of_simulations=40, 
    simulation_size=200000,
    tqdm=tqdm,
    )


In [9]:
# Before filtering best simulations

fig = plot([
        (stats.get_metric('deaths'), 'mortes'),
        (stats.get_metric('confirmed_deaths'), 'confirmadas'),
    ], 'Mortes - Real x Confirmadas - Cenário atual provável', False, stop=90)

fig  # Or fig.show() if just fig doesn't work

FigureWidget({
    'data': [{'fill': 'toself',
              'fillcolor': 'rgba(0,0,255,0.25)',
              …

In [10]:
# Filtering best simulations
stats.filter_best_scores(br_saopaulo_score_fn, 0.25)

fig = plot([
        (stats.get_metric('deaths'), 'mortes'),
        (stats.get_metric('confirmed_deaths'), 'confirmadas'),
    ], 'Mortes - Real x Confirmadas - Cenário atual provável', False, stop=90)

fig   # Or fig.show() if just fig doesn't work

FigureWidget({
    'data': [{'fill': 'toself',
              'fillcolor': 'rgba(0,0,255,0.25)',
              …